In [1]:
%load_ext autoreload
%autoreload 2

import os 
import shutil
import numpy as n
from datetime import date
from matplotlib import pyplot as plt
import time
import napari

In [2]:
os.chdir('/home/ali/packages/s2p-lbm/')

from suite3d.job import Job
from suite3d import lbmio, utils, ui
from suite3d import tiff_utils as tfu

In [5]:
# Create the job
job = Job('/mnt/md0/runs','Demo-GPU', create=False, overwrite=True, verbosity = 3)

   Found and loaded params from /mnt/md0/runs/s3d-Demo-GPU/params.npy


In [6]:
import cupy

In [7]:
import suite3d.svd_utils as svu

In [12]:
from dask import array as darr

In [16]:
job.params['svd_time_chunk']

4000

In [15]:
job.params['svd_pix_chunk']

80000

In [ ]:
import dask

In [28]:
block = n.random.normal(0, 1, (1000, 4*200*200))
block = darr.asarray(block)
block = block.rechunk((job.params['svd_time_chunk'], job.params['svd_pix_chunk']))

In [22]:
temps_cpu = run_svd_on_block(block, 200, save_zarr=False)

In [26]:
%%time
temps_cpu = dask.compute(temps_cpu)

CPU times: user 48.8 s, sys: 57.4 s, total: 1min 46s
Wall time: 2.08 s


In [39]:
block = n.random.normal(0, 1, (1000, 4*200*200))
block = darr.asarray(block)
block = block.rechunk((job.params['svd_time_chunk'], job.params['svd_pix_chunk']))

In [53]:
block_gpu = block_gpu.persist()

In [54]:
block_gpu = block.map_blocks(cupy.asarray)

In [55]:
u,s,v = darr.linalg.svd_compressed(block_gpu, k=200)

In [56]:
%%time
ug = u.compute()

TypeError: Argument 'b' has incorrect type (expected cupy._core.core._ndarray_base, got numpy.ndarray)

In [ ]:
temps_gpu = run_svd_on_block(block_gpu, 200, save_zarr=False)

In [46]:
%%time
temps_gpu = dask.compute(temps_gpu)

TypeError: Argument 'b' has incorrect type (expected cupy._core.core._ndarray_base, got numpy.ndarray)

In [9]:

def run_svd_on_block(block, n_comp, svd_dir=None, save_zarr=True, timechunk=200, compchunk=100,
                     svd_compressed_params = {}, use_regular_svd=False):
    if use_regular_svd:
        u,s,v = darr.linalg.svd(block)
    else:
        u,s,v = darr.linalg.svd_compressed(block, k=n_comp, **svd_compressed_params)
    if not save_zarr:
        return u,s,v
    u = u.rechunk((timechunk,compchunk))
    s = s.rechunk((compchunk,))
    v = v.rechunk((compchunk, -1))
    zarrs = {}
    temp_vals = []
    for zarr_name, arr in zip(['u','s','v'], (u,s,v)):
        zarr_path = os.path.join(svd_dir, '%s.zarr' % zarr_name)
        zarrs[zarr_name] = zarr.open(zarr_path, compressor=None, mode='w',
                                        shape=arr.shape, chunks=arr.chunksize, dtype=arr.dtype)
        temp_vals.append(arr.store(zarrs[zarr_name], compute=False, lock=False))
        
    return temp_vals


In [10]:

def run_svd_on_block_gpu(block, n_comp, svd_dir=None, save_zarr=True, timechunk=200, compchunk=100,
                     svd_compressed_params = {}, use_regular_svd=False):
    block = block.map_blocks(cupy.asarray)
    u,s,v = darr.linalg.svd_compressed(block, k=n_comp, **svd_compressed_params)
    if not save_zarr:
        return u,s,v
    u = u.rechunk((timechunk,compchunk))
    s = s.rechunk((compchunk,))
    v = v.rechunk((compchunk, -1))
    zarrs = {}
    temp_vals = []
    for zarr_name, arr in zip(['u','s','v'], (u,s,v)):
        zarr_path = os.path.join(svd_dir, '%s.zarr' % zarr_name)
        zarrs[zarr_name] = zarr.open(zarr_path, compressor=None, mode='w',
                                        shape=arr.shape, chunks=arr.chunksize, dtype=arr.dtype)
        temp_vals.append(arr.store(zarrs[zarr_name], compute=False, lock=False))
        
    return temp_vals
